# Tomography

## Imports

In [1]:
import tkinter as tk
from tkinter import Tk, LabelFrame, Label, Entry, Button, StringVar, OptionMenu
import matplotlib as plt
plt.use("TkAgg")
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
from matplotlib.patches import Ellipse
import numpy as np
from skimage.transform import radon, iradon, rescale

## GUI Callbacks

### GUI - helper functions

Input validation

In [2]:
ellipses = list()
    
def checker(a, c):
    def func(b):
        return a<=b and b<=c
    return func

def check(element, check_func):
    try:
        element = float(element)
        return check_func(element)
    except ValueError:
        return False

Phantom, Radon and Inverse Radon plots

In [3]:
# e = [intensity, inc, axis_x, axis_y, center_x, center_y]
# e =    e[0]    e[1]   e[2]    e[3]     e[4]      e[5]
def plot_phantom(ellipses, figure):
    figure.clear()
    for e in ellipses:
        ell = Ellipse(xy=(e[4], e[5]), width=e[2], height=e[3], angle=e[1])
        print('Artist: ', ell)
        figure.add_artist(ell)
        ell.set_clip_box(figure.bbox)
        ell.set_alpha(e[0])
#        ell.set_facecolor()

    figure.set_xlim(-1, 1)
    figure.set_ylim(-1, 1)
    canvas_p.draw()
    
def plot_rad(sinogram, figure):
    figure.clear()
    figure.imshow(sinogram, cmap=plt.cm.Greys_r)
    figure.set_xlim(0, 100)
    figure.set_ylim(0, 100)
    canvas_r.draw()

def plot_radon_inv(rad_inv, figure):
    figure.clear()
    figure.imshow(sinogram, cmap=plt.cm.Greys_r)
    figure.set_xlim(0, 100)
    figure.set_ylim(0, 100)
    canvas_r.draw()


### Callbacks

In [4]:
# Button Callbacks
def add_clicked():
    # Add ellipse to plot 
    checkers = [checker(-1, 1), checker(0, 359), checker(10e-9, 1), checker(10e-9, 1), checker(-1, 1), checker(-1, 1)]
    vals = [intensity, inc, axis_x, axis_y, center_x, center_y]
    vals = [i.get() for i in vals]
    if all([check(a, b) for a, b in zip(vals, checkers)]):
        vals = [float(i) for i in vals]
        print('add_clicked correct input')
        # add to ellipses
        ellipses.append(vals)
        print(ellipses)
    else:
        print('add_clicked incorrect input')
    # Plot new graph
    plot_phantom(ellipses, phantom_fig)
        
# Intensidad: Varía entre -1 y 1, ya que es aditiva. (I)
# Inclinación: Ángulo en grados de rotación respecto al eje x. (A)
# Semi-eje X: Valor del semi-eje X de la elipse, entre (0 ; 1]. (X)
# Semi-eje Y: Valor del semi-eje Y de la elipse, entre (0 ; 1]. (Y)
# Centro X: Coordenada X del centro de la elipse, entre [-1 ; 1]. (CX)
# Centro Y: Coordenada Y del centro de la elipse, entre [-1 ; 1]. (CY)

def rem_clicked(): 
    # Remove last plotted ellipse
    if ellipses:
        ellipses.pop()
        print(ellipses)
    # Plot new graph
    plot_phantom(ellipses=ellipses, figure=phantom_fig)

def see_clicked(): 
    # Parse
    checkers = [checker(0, 180), checker(1, 180), checker(0, 180), checker(0, 180)]

    vals = [fro, to, step, angle]
    vals = [i.get() for i in vals]
    
    if all([check(a, b) for a, b in zip(vals, checkers)]):
        print('see_clicked correct input')
        vals = [float(val) for val in vals]
        fro_, to_, step_, angle_ = vals
        
        print('Vals (fro, to, step, angle): ', vals)
        if fro_ < to_ and step_ <= (to_-fro_) and angle_<to_ and angle_>fro_:
            vals = [float(i) for i in vals]
            print('\see_clicked correct intervals')
            # Calculate radon
            thetas = [angle_] # https://stackoverflow.com/questions/35355930/matplotlib-figure-to-image-as-a-numpy-array
            canvas_p.draw()
            width, height = f_1.get_size_inches() * f_1.get_dpi()
            width, height = int(width), int(height)
            
            image = np.frombuffer(canvas_p.tostring_rgb(), dtype='uint8').reshape(height, width, 3)
            
            image = np.dot(image[...,:3], [0.2989, 0.5870, 0.1140])
#             image = np.asarray(image, dtype='uint8')
#             image = image[,50: ,50]
#             image = rescale(image, scale=0.4, mode='reflect', multichannel=False)
#             plt.imshow(gray, cmap=plt.get_cmap('gray'), vmin=0, vmax=1)
            sinogram = radon(image, theta=thetas, circle=True)
            # Plot result
            plot_rad(sinogram=sinogram, figure=rad_fig)
        else:
            print('see_clicked incorrect input')
    
def calc_rad_clicked(): 

    # Parse
    checkers = [checker(0, 180), checker(1, 180), checker(0, 180)]

    vals = [fro, to, step]
    vals = [i.get() for i in vals]
    
    if all([check(a, b) for a, b in zip(vals, checkers)]):
        print('calc_rad_clicked correct input')
        vals = [float(val) for val in vals]
        fro_, to_, step_ = vals
        
        print('Vals (fro, to, step, angle): ', vals)
        if fro_ < to_ and step_ <= (to_-fro_):
            vals = [float(i) for i in vals]
            print('\tcalc_rad_clicked correct intervals')
            # Calculate radon
            thetas = [fro_ + i*step_ for i in range(int((to_-fro_)/step_))]
            
            # https://stackoverflow.com/questions/35355930/matplotlib-figure-to-image-as-a-numpy-array
            canvas_p.draw()
            width, height = f_1.get_size_inches() * f_1.get_dpi()
            width, height = int(width), int(height)
            
            image = np.frombuffer(canvas_p.tostring_rgb(), dtype='uint8').reshape(height, width, 3)
            
            image = np.dot(image[...,:3], [0.2989, 0.5870, 0.1140])
#             image = np.asarray(image, dtype='uint8')
#             image = image[,50: ,50]
#             image = rescale(image, scale=0.4, mode='reflect', multichannel=False)
#             plt.imshow(gray, cmap=plt.get_cmap('gray'), vmin=0, vmax=1)
            print(image)
            print(image.shape)
            sinogram = radon(image, theta=thetas, circle=True)
            # Plot result
            plot_rad(sinogram=sinogram, figure=rad_fig)
        else:
            print('calc_rad_inv_clicked incorrect input')

def calc_rad_inv_clicked(): 
 # Parse
    checkers = [checker(0, 180), checker(1, 180), checker(0, 180)]

    vals = [fro, to, step]
    vals = [i.get() for i in vals]
    
    if all([check(a, b) for a, b in zip(vals, checkers)]):
        print('calc_rad_inv_clicked correct input')
        vals = [float(val) for val in vals]
        fro_, to_, step_ = vals
        
        print('Vals (fro, to, step, angle): ', vals)
        if fro_ < to_ and step_ <= (to_-fro_):
            vals = [float(i) for i in vals]
            print('\calc_rad_inv_clicked correct intervals')
            # Calculate radon
            thetas = [fro_ + i*step_ for i in range(int((to_-fro_)/step_))]
            
            # https://stackoverflow.com/questions/35355930/matplotlib-figure-to-image-as-a-numpy-array
            canvas_p.draw()
            width, height = f_1.get_size_inches() * f_1.get_dpi()
            width, height = int(width), int(height)
            
            image = np.frombuffer(canvas_p.tostring_rgb(), dtype='uint8').reshape(height, width, 3)
            
            image = np.dot(image[...,:3], [0.2989, 0.5870, 0.1140])
#             image = np.asarray(image, dtype='uint8')
#             image = image[,50: ,50]
#             image = rescale(image, scale=0.4, mode='reflect', multichannel=False)
#             plt.imshow(gray, cmap=plt.get_cmap('gray'), vmin=0, vmax=1)
            print(image)
            print(image.shape)
            sinogram = iradon(image, theta=thetas, circle=True)
            # Plot result
            plot_radon_inv(rad_inv=sinogram, figure=rad_fig)
        else:
            print('calc_rad_inv_clicked incorrect input')

## GUI Definition

In [5]:
# Figures
f_1 = Figure(figsize=(2,2), dpi=100)
phantom_fig = f_1.add_subplot(111)
# phantom_fig.plot([1,2,3,4,5,6,7,8],[5,6,1,3,8,9,3,5])

f_2 = Figure(figsize=(3,3), dpi=100)
rad_fig = f_2.add_subplot(111)
# rad_fig.plot([1,2,3,4,5,6,7,8],[5,6,1,3,8,9,3,5])

f_3 = Figure(figsize=(3,3), dpi=100)
rad_inv_fig = f_3.add_subplot(111)
rad_inv_fig.plot([1,2,3,4,5,6,7,8],[5,6,1,3,8,9,3,5])

In [6]:
# Create root window 
root = tk.Tk() 
  
# Root window title and dimension 
root.title("Tomography Simulator") 
root.geometry('1080x640') 

# Widgets 

# Label Frames 

# Phantom
label_frame_phantom = LabelFrame(root, text = 'Phantom') 
label_frame_phantom.pack(expand = 'yes', fill = 'both', side = tk.LEFT)
label_frame_add = LabelFrame(label_frame_phantom, text = 'Add ellipse') 
label_frame_add.pack()
  
Label(label_frame_add, text = 'Intensity').grid(column=0, row=0)  
Label(label_frame_add, text = 'Inclination').grid(column=0, row=1)
Label(label_frame_add, text = 'Semi-axis X').grid(column=0, row=2)
Label(label_frame_add, text = 'Semi-axis Y').grid(column=0, row=3)
Label(label_frame_add, text = 'Center X').grid(column=0, row=4)
Label(label_frame_add, text = 'Center Y').grid(column=0, row=5)

intensity = Entry(label_frame_add, width=10) 
intensity.grid(column=1, row=0)
inc = Entry(label_frame_add, width=10) 
inc.grid(column=1, row=1)
axis_x = Entry(label_frame_add, width=10) 
axis_x.grid(column=1, row=2)
axis_y = Entry(label_frame_add, width=10) 
axis_y.grid(column=1, row=3)
center_x = Entry(label_frame_add, width=10) 
center_x.grid(column=1, row=4)
center_y = Entry(label_frame_add, width=10) 
center_y.grid(column=1, row=5)

# Radon Frame
label_frame_radon = LabelFrame(root, text = 'Radon Transform') 
label_frame_radon.pack(expand = 'yes', fill = 'both', side = tk.RIGHT)

label_frame_rad = LabelFrame(label_frame_radon, text = 'Radon') 
label_frame_rad.pack(expand = 'yes', fill = 'both', side = tk.TOP)

label_frame_rad_inv = LabelFrame(label_frame_rad, text = 'Radon Inverse') 
label_frame_rad_inv.pack(expand = 'yes', fill = 'both', side = tk.BOTTOM)


# Radon
label_frame_r_config = LabelFrame(label_frame_rad, text = 'Config') 
label_frame_r_config.pack(side = tk.LEFT)

Label(label_frame_r_config, text = 'From').grid(column=0, row=0)
Label(label_frame_r_config, text = 'Step').grid(column=0, row=1)
Label(label_frame_r_config, text = 'To').grid(column=0, row=2)
Label(label_frame_r_config, text = 'Angle').grid(column=0, row=4)

fro = Entry(label_frame_r_config, width=10) 
fro.grid(column=1, row=0)
step = Entry(label_frame_r_config, width=10) 
step.grid(column=1, row=1)
to = Entry(label_frame_r_config, width=10) 
to.grid(column=1, row=2)
angle = Entry(label_frame_r_config, width=10) 
angle.grid(column=1, row=4)

# Radon Inv
label_frame_ri_config = LabelFrame(label_frame_rad_inv, text = 'Config') 
label_frame_ri_config.pack(side = tk.LEFT)

Label(label_frame_ri_config, text = 'From').grid(column=0, row=0)
Label(label_frame_ri_config, text = 'Step').grid(column=0, row=1)
Label(label_frame_ri_config, text = 'To').grid(column=0, row=2)
Label(label_frame_ri_config, text = 'Interpolation').grid(column=0, row=3)
Label(label_frame_ri_config, text = 'Filter').grid(column=0, row=4)

fro_i = Entry(label_frame_ri_config, width=10) 
fro_i.grid(column=1, row=0)
step_i = Entry(label_frame_ri_config, width=10) 
step_i.grid(column=1, row=1)
to_i = Entry(label_frame_ri_config, width=10) 
to_i.grid(column=1, row=2)

var_int = StringVar(label_frame_ri_config)
var_int.set("No selection") # default value
var_filt = StringVar(label_frame_ri_config)
var_filt.set("No Selection") # default value

interp = OptionMenu(label_frame_ri_config, var_int, "one", "two", "three")
interp.grid(column=1, row=3)
interp = OptionMenu(label_frame_ri_config, var_filt, "one", "two", "three")
interp.grid(column=1, row=4)

# Canvas
canvas_p = FigureCanvasTkAgg(f_1, label_frame_phantom)
canvas_p.get_tk_widget().pack(side=tk.BOTTOM, fill=tk.BOTH, expand=True)

canvas_r = FigureCanvasTkAgg(f_2, label_frame_rad)
canvas_r.get_tk_widget().pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

canvas_ri = FigureCanvasTkAgg(f_3, label_frame_rad_inv)
canvas_ri.get_tk_widget().pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

# Button widgets
btn_add = Button(label_frame_add, text = "Add", fg = "red", command=add_clicked)
btn_rem = Button(label_frame_add, text = "Remove Last", fg = "red", command=rem_clicked)
  
btn_add.grid(column=0, row=6)
btn_rem.grid(column=1, row=6)

btn_see = Button(label_frame_r_config, text = "See", fg = "red", command=see_clicked)
btn_calc = Button(label_frame_r_config, text = "Calculate", fg = "red", command=calc_rad_clicked)

btn_calc.grid(column=1, row=3)
btn_see.grid(column=1, row=5)

btn_calc_inv = Button(label_frame_ri_config, text = "Calculate", fg = "red", command=calc_rad_inv_clicked)
btn_calc_inv.grid(column=1, row=5)

# Run Mainloop
root.mainloop()

add_clicked correct input
[[0.2, 0.2, 0.2, 0.2, 0.0, 0.0]]
Artist:  Ellipse(xy=(0.0, 0.0), width=0.2, height=0.2, angle=0.2)
calc_rad_clicked correct input
Vals (fro, to, step, angle):  [0.0, 180.0, 10.0]
	calc_rad_clicked correct intervals
[[254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]
 [254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]
 [254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]
 ...
 [254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]
 [254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]
 [254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]]
(450, 387)


c:\users\tomas\appdata\local\programs\python\python38\lib\site-packages\skimage\transform\radon_transform.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  coords = np.array(np.ogrid[:image.shape[0], :image.shape[1]])
c:\users\tomas\appdata\local\programs\python\python38\lib\site-packages\skimage\transform\radon_transform.py:91: UserWarning: Radon transform: image must be zero outside the reconstruction circle
  warn('Radon transform: image must be zero outside the '


calc_rad_inv_clicked correct input
Vals (fro, to, step, angle):  [0.0, 180.0, 10.0]
\calc_rad_inv_clicked correct intervals
[[254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]
 [254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]
 [254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]
 ...
 [254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]
 [254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]
 [254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]]
(450, 409)


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\tomas\appdata\local\programs\python\python38\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-4-c254e6432e8b>", line 141, in calc_rad_inv_clicked
    sinogram = iradon(image, theta=thetas, circle=True)
  File "c:\users\tomas\appdata\local\programs\python\python38\lib\site-packages\skimage\_shared\utils.py", line 114, in fixed_func
    return func(*args, **kwargs)
  File "c:\users\tomas\appdata\local\programs\python\python38\lib\site-packages\skimage\transform\radon_transform.py", line 260, in iradon
    raise ValueError("The given ``theta`` does not match the number of "
ValueError: The given ``theta`` does not match the number of projections in ``radon_image``.


calc_rad_inv_clicked correct input
Vals (fro, to, step, angle):  [0.0, 180.0, 10.0]
\calc_rad_inv_clicked correct intervals
[[254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]
 [254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]
 [254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]
 ...
 [254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]
 [254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]
 [254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]]
(450, 409)
calc_rad_inv_clicked correct input
Vals (fro, to, step, angle):  [0.0, 180.0, 10.0]
\calc_rad_inv_clicked correct intervals
[[254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]
 [254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]
 [254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]
 ...
 [254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]
 [254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]
 [254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745

...

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\tomas\appdata\local\programs\python\python38\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-4-c254e6432e8b>", line 141, in calc_rad_inv_clicked
    sinogram = iradon(image, theta=thetas, circle=True)
  File "c:\users\tomas\appdata\local\programs\python\python38\lib\site-packages\skimage\_shared\utils.py", line 114, in fixed_func
    return func(*args, **kwargs)
  File "c:\users\tomas\appdata\local\programs\python\python38\lib\site-packages\skimage\transform\radon_transform.py", line 260, in iradon
    raise ValueError("The given ``theta`` does not match the number of "
ValueError: The given ``theta`` does not match the number of projections in ``radon_image``.


calc_rad_inv_clicked correct input
Vals (fro, to, step, angle):  [0.0, 180.0, 10.0]
\calc_rad_inv_clicked correct intervals
[[254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]
 [254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]
 [254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]
 ...
 [254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]
 [254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]
 [254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]]
(450, 409)


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\tomas\appdata\local\programs\python\python38\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-4-c254e6432e8b>", line 141, in calc_rad_inv_clicked
    sinogram = iradon(image, theta=thetas, circle=True)
  File "c:\users\tomas\appdata\local\programs\python\python38\lib\site-packages\skimage\_shared\utils.py", line 114, in fixed_func
    return func(*args, **kwargs)
  File "c:\users\tomas\appdata\local\programs\python\python38\lib\site-packages\skimage\transform\radon_transform.py", line 260, in iradon
    raise ValueError("The given ``theta`` does not match the number of "
ValueError: The given ``theta`` does not match the number of projections in ``radon_image``.


calc_rad_inv_clicked correct input
Vals (fro, to, step, angle):  [0.0, 180.0, 10.0]
\calc_rad_inv_clicked correct intervals
[[254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]
 [254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]
 [254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]
 ...
 [254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]
 [254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]
 [254.9745 254.9745 254.9745 ... 254.9745 254.9745 254.9745]]
(450, 409)


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\tomas\appdata\local\programs\python\python38\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-4-c254e6432e8b>", line 141, in calc_rad_inv_clicked
    sinogram = iradon(image, theta=thetas, circle=True)
  File "c:\users\tomas\appdata\local\programs\python\python38\lib\site-packages\skimage\_shared\utils.py", line 114, in fixed_func
    return func(*args, **kwargs)
  File "c:\users\tomas\appdata\local\programs\python\python38\lib\site-packages\skimage\transform\radon_transform.py", line 260, in iradon
    raise ValueError("The given ``theta`` does not match the number of "
ValueError: The given ``theta`` does not match the number of projections in ``radon_image``.
